In [38]:
#from blackbox import BlackBox
import random
import binascii
from collections import defaultdict
import statistics

In [3]:
class BlackBox:

    def ask(self, file, num):
        lines = open(file,'r').readlines()
        users = [0 for i in range(num)]
        for i in range(num):
            users[i] = lines[random.randint(0, len(lines) - 1)].rstrip("\n")
        return users

In [4]:
def myhashs(s):
    result = []
    for f in hash_fun_list:
        result.append(f(s))
    return result

In [5]:
def str_to_int(string):
    return int(binascii.hexlify(string.encode('utf8')),16)

In [6]:
def create_hash_function_list(no_of_hash):
    m = 69997
    prime = 9999999851
    hash_a = random.sample(range(1,10000000000),no_of_hash)
    hash_b = random.sample(range(1,10000000000),no_of_hash)
    global hash_fun_list
    hash_fun_list = []
    for i in range(no_of_hash):
        a = hash_a[i]
        b = hash_b[i]
        hash_fun_list.append(lambda x, a=a, b=b, prime=prime :  ((a*str_to_int(x) + b) % prime) % 69997)

    return hash_fun_list


In [7]:
filter_bit_array = [0 for _ in range(69997)]

In [55]:
stream_size = 300
no_of_asks = 30
input_file = "users.txt"
output_file = "output.csv"

no_of_hash_funcs = 200
no_of_groups = 40

hash_group_size = int(no_of_hash_funcs / no_of_groups)

In [56]:
bx = BlackBox()
create_hash_function_list(no_of_hash_funcs)

[<function __main__.create_hash_function_list.<locals>.<lambda>(x, a=7870356510, b=7878843311, prime=9999999851)>,
 <function __main__.create_hash_function_list.<locals>.<lambda>(x, a=3967936108, b=6635995137, prime=9999999851)>,
 <function __main__.create_hash_function_list.<locals>.<lambda>(x, a=8143607030, b=3464593657, prime=9999999851)>,
 <function __main__.create_hash_function_list.<locals>.<lambda>(x, a=3276504435, b=7482986680, prime=9999999851)>,
 <function __main__.create_hash_function_list.<locals>.<lambda>(x, a=7610123365, b=8505029333, prime=9999999851)>,
 <function __main__.create_hash_function_list.<locals>.<lambda>(x, a=7305932976, b=2898937800, prime=9999999851)>,
 <function __main__.create_hash_function_list.<locals>.<lambda>(x, a=8162412587, b=1227430331, prime=9999999851)>,
 <function __main__.create_hash_function_list.<locals>.<lambda>(x, a=4220901271, b=658525445, prime=9999999851)>,
 <function __main__.create_hash_function_list.<locals>.<lambda>(x, a=3479772183, 

In [59]:
output = []
for i in range(no_of_asks):
    stream_users = bx.ask(input_file, stream_size)
    unique_users = defaultdict(int)
    count_of_trailing = [0 for _ in range(no_of_hash_funcs)]
    for user in stream_users:
        unique_users[user] = 1
        hashes = myhashs(user)
        for i in range(len(hashes)):
            z = bin(hashes[i])

            zeros = 0
            for j in range(-1, -len(z), -1):
                if z[j]=='0':
                    zeros+=1
                else:
                    break
            count_of_trailing[i] = max(count_of_trailing[i], zeros)
    R_vals = []
    for i in range(no_of_groups):
        avg_R = count_of_trailing[i*hash_group_size:i*hash_group_size+hash_group_size]
        R_vals.append(sum(avg_R)/len(avg_R))
    median = statistics.median(R_vals)
    estimate_users = round(pow(2,median))
    actual_users = len(unique_users.keys())
    #print(no_of_asks, actual_users,estimate_users)
    output.append([no_of_asks, actual_users,estimate_users])

In [62]:
with open(output_file,'w+') as f:
    f.write('Time,Ground Truth,Estimation\n')
    for i in output:
        f.write(str(i[0]) + "," + str(i[1])+ "," + str(i[2]) + "\n")